In [66]:
import os
import pandas as pd
import numpy as np
from scipy import spatial
from sklearn.cluster import KMeans
from collections import Counter
from sko.GA import GA_TSP
import pickle
import timeit
import random

def files(path):
    flist = []
    for file in os.listdir(path):
        if file.endswith(".txt"):
            flist.append(file)
    return flist
def transformInstances():
    pwd = os.getcwd()
    all_instances = files(pwd)

    for instance in all_instances:
        print("Working on {}".format(instance))
        filepath = instance
        customer_numbers = []
        xs = []
        ys = []
        demands = []

        with open(filepath) as fp:
            number_of_customers, best_known_solution_value = fp.readline().split()
            vehicle_capacites = fp.readline()
            xdepot, ydepot = fp.readline().split()
            for i in range(int(number_of_customers)):
                line = fp.readline().split()
                customer_numbers.append(line[0])
                xs.append(line[1])
                ys.append(line[2])
                demands.append(line[3])

        Name = filepath.partition(".")[0].upper()

        with open('{}.vrp'.format(Name), 'w') as fp:
            fp.write('NAME: {}\n'.format(Name))
            fp.write('BEST_KNOWN: {}\n'.format(best_known_solution_value))
            fp.write('COMMENT: {}\n'.format(best_known_solution_value + '0000'))
            fp.write('DIMENSION: {}\n'.format(str(int(number_of_customers) + 1)))
            fp.write('CAPACITY: {}'.format(vehicle_capacites))
            fp.write('EDGE_WEIGHT_FORMAT: FUNCTION\n')
            fp.write('EDGE_WEIGHT_TYPE: EXACT_2D\n')
            fp.write('NODE_COORD_SECTION\n')
            fp.write('1 0 0\n')
            for i in range(int(number_of_customers)):
                fp.write('{} {} {}\n'.format(str(int(customer_numbers[i])+1),xs[i],ys[i]))
            fp.write('DEMAND_SECTION\n')
            fp.write('1 0\n')
            for i in range(int(number_of_customers)):
                fp.write('{} {}\n'.format(str(int(customer_numbers[i])+1),demands[i]))
            fp.write('DEPOT_SECTION\n')
            fp.write('1\n')
            fp.write('-1\n')
            fp.write('EOF\n')
def createDistMatrix(xy):
    distance_matrix = spatial.distance.cdist(xy, xy, metric='euclidean')
    return distance_matrix
def createCluster(xydf, nTruck):
    if(nTruck<len(xydf)):
        kmeans = KMeans(n_clusters=nTruck, random_state=0).fit(xydf)
        xydf['cluster'] = kmeans.labels_
        keys = list(Counter(xydf.cluster).keys())
        values = list(Counter(xydf.cluster).values())
        flag = True
        return (xydf,flag)
    else:
        flag = False
        return (xydf,flag)
def findSolution(distance_matrix, num_points):
    def cal_total_distance(routine):
        '''The objective function. input routine, return total distance.
        cal_total_distance(np.arange(num_points))
        '''
        num_points, = routine.shape
        return sum([distance_matrix[routine[i % num_points], routine[(i + 1) % num_points]] for i in range(num_points)])
    ga_tsp = GA_TSP(func=cal_total_distance, n_dim=num_points, size_pop=20, max_iter=500, prob_mut=1)
    best_points, best_distance = ga_tsp.run()
    return (best_points, best_distance)
def createSolDisIt(nTruck, xydf, df):
    Solutions = [[] for i in range(nTruck)]
    Distances = [[] for i in range(nTruck)]
    Items = [[] for i in range(nTruck)]
    for i in range(len(Solutions)):
        cdf = xydf.loc[xydf['cluster'] == i]
        icdf = cdf.index.tolist()
        x = df.iloc[icdf, icdf]
        Items[i] = x.index.tolist()
        x = x.to_numpy()
        best_points, best_distance = findSolution(x,len(x))
        Solutions[i] = list(best_points)
        Distances[i] = float(best_distance)
        for j in range(len(Solutions[i])):
            Solutions[i][j] = Items[i][Solutions[i][j]]
        Distances[i] += df[0][Solutions[i][0]]
        Solutions[i].insert(0,0)
    return(Solutions,Distances)
def makeMove(curt_sol, rsol, current, dmatrix, Visited):
    move = curt_sol.pop(0)
    Visited.append(move)
    rdis = dmatrix[current][move]
    rsol.append(move)
    return (move,rsol,rdis, Visited)
def createTest(customer_numbers, demands, xy, nTruck):
    distance_matrix = createDistMatrix(xy)
    df = pd.DataFrame(data=distance_matrix)
    xydf = pd.DataFrame(data=xy, columns = ['X', 'Y'])
    xydf['index1'] = xydf.index
    xydf = xydf.iloc[1:]
    xydf,flag = createCluster(xydf,nTruck)
    Solutions,Distances = createSolDisIt(nTruck, xydf, df)
    Rsol = [[] for i in range(nTruck)]
    Rdis = [0 for i in range(nTruck)]
    Rcap = [0 for i in range(nTruck)]
    Rcurt = [0 for i in range(nTruck)]
    Visited = []
    for index in range(len(Solutions)):
        for i in range(24):
            if (i == 12 or i == 13):
                if(len(Solutions[index])>0):
                    move,rsol, rdis, Visited = makeMove(Solutions[index],Rsol[index],Rcurt[index], distance_matrix, Visited)
                    Rcap[index] += demands[Rcurt[index]]
                    Rcurt[index] = move
                    Rdis[index] += rdis
        while(len(Solutions[index])>0):
            move,rsol, rdis, Visited = makeMove(Solutions[index],Rsol[index],Rcurt[index], distance_matrix, Visited)
            Rcap[index] += demands[Rcurt[index]]
            Rcurt[index] = move
            Rdis[index] += rdis
    newxy = np.empty([1, 2], dtype=int)
    newxy[0][0] = 1
    newxy[0][1] = 2
    xy = np.vstack((xy, newxy))
    demands.append(10)
    distance_matrix = createDistMatrix(xy)
    random.seed(0)
    for i,n in enumerate(distance_matrix):
        for j,m in enumerate(n):
            if distance_matrix[i][j] != 0:
                r = random.randint(0, 2)
                p = random.randint(0, 10)
                if r ==1:
                    distance_matrix[i][j] += p
                elif r ==2:
                    distance_matrix[i][j] -= p
    df = pd.DataFrame(data=distance_matrix)
    xydf = pd.DataFrame(data=xy, columns = ['X', 'Y'])
    xydf['index1'] = xydf.index
    xydf = xydf.iloc[1:]
    for i in range(len(Visited)):
        xydf = xydf.loc[xydf['index1'] != Visited[i]]
    xydf,flag = createCluster(xydf,nTruck)
    if(flag==False):
        newrows = xydf.index1.values.tolist()
        for value in newrows:
            Solutions[Rcap.index(min(Rcap))].append(value)
    else:
        Solutions,Distances = createSolDisIt(nTruck, xydf, df)
    for index in range(len(Solutions)):
        for i in range(24):
            if (i == 12 or i == 13):
                if(len(Solutions[index])>0):
                    move,rsol, rdis, Visited = makeMove(Solutions[index],Rsol[index],Rcurt[index], distance_matrix, Visited)
                    rdis += 15
                    Rcap[index] += demands[Rcurt[index]]
                    Rcurt[index] = move
                    Rdis[index] += rdis
        while(len(Solutions[index])>0):
            move,rsol, rdis, Visited = makeMove(Solutions[index],Rsol[index],Rcurt[index], distance_matrix, Visited)
            Rcap[index] += demands[Rcurt[index]]
            Rcurt[index] = move
            Rdis[index] += rdis
    for index in range(len(Solutions)):
        Rdis[index] +=distance_matrix[Rcurt[index]][0]
    return (Rsol, Rdis, Rcap)
def loadFile(filepath):
    customer_numbers = []
    demands = []
    with open(filepath) as fp:
        name = fp.readline().split()[1]
        best_know = float(fp.readline().split()[1])
        fp.readline().split()
        dimension = int(fp.readline().split()[1])
        xy = np.empty([dimension, 2], dtype=int)
        capacity = int(fp.readline().split()[1])
        fp.readline().split()
        fp.readline().split()
        test = fp.readline().split()
        for i in range(dimension):
            line = fp.readline().split()
            customer_numbers.append(line[0])
            xy[i][0] = int(line[1])
            xy[i][1] = int(line[2])
        fp.readline().split()
        for i in range(dimension):
            line = int(fp.readline().split()[1])
            demands.append(line)
    return (customer_numbers, demands, xy)
def createGml(customer_numbers,xy, Rsol, filename):
    newxy = np.empty([1, 2], dtype=int)
    newxy[0][0] = 1
    newxy[0][1] = 2
    xy = np.vstack((xy, newxy))
    for i, sol in enumerate(Rsol):
        if Rsol[i][-1] != 0:
            Rsol[i].append(0)
    customer_numbers.append(len(customer_numbers)+1)
    with open('{}.gml'.format(filename), 'w') as fp:
        fp.write('graph [ hierarchic 1 directed 1\n')
        for i, n in enumerate(xy):
            fp.write('node [ id {} graphics [ x {} y {} w 1  h 1 type "roundrectangle"] LabelGraphics [text  "{}" fontSize 1 ] ]\n'.format(str(int(customer_numbers[i])-1),xy[i][0],xy[i][1],str(int(customer_numbers[i])-1)))
        for m, sol in enumerate(Rsol):
            for i, j in enumerate(sol[:-1]):
                if m == 1:
                    fp.write('edge [ source  {} target {} graphics [ fill "#c01717" ] ]\n'.format(sol[i],sol[i+1]))
                elif m == 2:
                    fp.write('edge [ source  {} target {} graphics [ fill "#000000" ] ]\n'.format(sol[i],sol[i+1]))
                else:
                    fp.write('edge [ source  {} target {} graphics [ fill "#0000FF" ] ]\n'.format(sol[i],sol[i+1]))
        fp.write(']\n')

In [70]:
ListofNames = ['F71.vrp',
               'C50.vrp',
               'C75.vrp',
               'C100.vrp',
               'C100B.vrp',
               'C120.vrp',
               'C150.vrp',
               'C199.vrp',
               'TAI75A.vrp',
               'TAI75B.vrp',
               'TAI75C.vrp',
               'TAI75D.vrp',
               'TAI100A.vrp',
               'TAI100B.vrp',
               'TAI100C.vrp',
               'TAI100D.vrp',
               'TAI150A.vrp',
               'TAI150B.vrp',
               'TAI150C.vrp',
               'TAI150D.vrp',]
for filepath in ListofNames:
    customer_numbers, demands, xy = loadFile(filepath)
    Results_RS = []
    Results_RD = []
    Results_RC = []
    Results_Ti = []
    for i in range(25):
        start_time = timeit.default_timer()
        Rsol, Rdis, Rcap = createTest(customer_numbers, demands, xy, 3)
        elapsed = timeit.default_timer() - start_time
        Results_RS.append(Rsol)
        Results_RD.append(Rdis)
        Results_RC.append(Rcap)
        Results_Ti.append(elapsed)
    ARRRD = np.asarray(Results_RD)
    ARRRC = np.asarray(Results_RC)
    ARRTI = np.asarray(Results_Ti)
    strname = filepath+'_RS'
    with open(strname, 'wb') as f:
        pickle.dump(Results_RS, f)
    strname = filepath+'_RD'
    with open(strname, 'wb') as f:
        pickle.dump(Results_RD, f)
    strname = filepath+'_RC'
    with open(strname, 'wb') as f:
        pickle.dump(Results_RC, f)
    strname = filepath+'_TI'
    with open(strname, 'wb') as f:
        pickle.dump(Results_Ti, f)
    print('filename =',filepath,'mean=',np.mean(ARRRD),'lowest value =',np.amin(ARRRD),'std=',np.std(ARRRD),'time=',np.mean(ARRTI))

filename = F71.vrp mean= 121.46016840076821 lowest value = 86.2176871036507 std= 16.091532911087143 time= 2.3671698760000437
filename = C50.vrp mean= 273.4661421811511 lowest value = 196.40700842509534 std= 38.71210729090401 time= 1.9428882199999498
filename = C75.vrp mean= 373.3244281382868 lowest value = 267.03969052265364 std= 56.77977676881476 time= 2.4058439919999364
filename = C100.vrp mean= 403.48049847673826 lowest value = 342.20412845269254 std= 31.201464330064795 time= 2.8684756879998896
filename = C100B.vrp mean= 313.58366167973423 lowest value = 196.25294427969567 std= 69.49834086074264 time= 2.8907280799999717
filename = C120.vrp mean= 356.5722135224902 lowest value = 229.57857777584326 std= 93.48443176933634 time= 3.3257027520001428
filename = C150.vrp mean= 553.7010417426499 lowest value = 424.29161034684597 std= 56.078375684462586 time= 3.885216224000178
filename = C199.vrp mean= 685.0668281155354 lowest value = 589.1154954620906 std= 47.09163788317247 time= 4.829618048

In [71]:
createGml(customer_numbers,xy,Rsol,'Migrafico')